# import

In [1]:
import pandas as pd
import numpy as np
import os, sys
from keras import models
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

sys.path.insert(0, os.getcwd())
print(sys.path)
from models import reports_gen_view as reports 

2025-02-21 14:11:09.461382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-21 14:11:09.461407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-21 14:11:09.462225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 14:11:09.466792: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-21 14:11:10.032099: W tensorflow/compiler/tf2

['/media/jczars/4C22F02A22F01B22/Pollen-Classification-CBAM', '/home/jczars/anaconda3/envs/tf/lib/python310.zip', '/home/jczars/anaconda3/envs/tf/lib/python3.10', '/home/jczars/anaconda3/envs/tf/lib/python3.10/lib-dynload', '', '/home/jczars/.local/lib/python3.10/site-packages', '/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages', '/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages/setuptools/_vendor']


# params

In [2]:
params = {
    'path_model':"results/phase1/AT_densenet+cbam_exp/0_DenseNet201/",
    'path_full_labels':"/media/jczars/4C22F02A22F01B22/$WinREAgent/Pollen_classification_view/BD/CPD1_Cr_Rs_400/Test/k1",
    'path_test': "BD/CPD1_TEST_190225/",
    'save_dir': "results/phase3/",
    'size': 224
    }

# run

In [3]:

print(f"params: {params}")
#listar arquivos
categories = sorted(os.listdir(params['path_full_labels']))
print(f"total categories: {len(categories)}, categories: {categories}") 

params: {'path_model': 'results/phase1/AT_densenet+cbam_exp/0_DenseNet201/', 'path_full_labels': '/media/jczars/4C22F02A22F01B22/$WinREAgent/Pollen_classification_view/BD/CPD1_Cr_Rs_400/Test/k1', 'path_test': 'BD/CPD1_TEST_190225/', 'save_dir': 'results/phase3/', 'size': 224}
total categories: 20, categories: ['asphodelus', 'calicotome', 'castanea', 'ceratonia', 'ebenus', 'erica', 'eucalyptus', 'ferula', 'myrtus', 'olea', 'origanum', 'oxalis', 'pinus', 'pistacia', 'salvia', 'satureja', 'sinapis', 'thymbra', 'urginea', 'vitis']


In [4]:
#create folders
k=1
folder=f"{params['save_dir']}k{k}"
print(f"folder: {folder}")
os.makedirs(folder, exist_ok=True) 

# Load model
print(f"\n[INFO] load model")
path_model = params['path_model']
nm_model= path_model.split("/")[-2]
path = f"{path_model}{nm_model}_bestLoss_{k}.keras"
print(f"\n[INFO] path_model: {path}\n")
model=None
model = models.load_model(path)
model.summary()  # Print model summary

folder: results/phase3/k1

[INFO] load model

[INFO] path_model: results/phase1/AT_densenet+cbam_exp/0_DenseNet201/0_DenseNet201_bestLoss_1.keras



2025-02-21 14:11:11.503992: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-21 14:11:11.531541: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-21 14:11:11.531737: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

In [5]:
# Equatorial views
size=params['size']
input_size = (size, size)
output_csv = f"{folder}class_reports_equatorial_k{k}.csv"
output_image= f"{folder}confusion_matrix_equatorial_k{k}.png" 

path_test=f"{params['path_test']}k{k}/"
equatorial_dir = f"{path_test}EQUATORIAL/" 
print(f"equatorial_dir: {equatorial_dir}")

test_generator = reports.load_data_test_vw(equatorial_dir, input_size)

equatorial_dir: BD/CPD1_TEST_190225/k1/EQUATORIAL/
Found 210 images belonging to 16 classes.


In [8]:
y_true = test_generator.classes  
print(f"Size y_true: {len(y_true)}") 
print(y_true)   

Size y_true: 210
[ 0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2
  2  2  2  2  2  2  2  2  2  3  3  3  4  5  6  6  6  6  6  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9 10 10 10 10 10 10 10
 10 10 10 11 11 11 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]


In [9]:
test_class_names = list(test_generator.class_indices.keys()) 
print(f"Size test_class_names: {len(test_class_names)}")
print(test_class_names)

Size test_class_names: 16
['asphodelus', 'castanea', 'ceratonia', 'ebenus', 'erica', 'eucalyptus', 'ferula', 'olea', 'origanum', 'pistacia', 'salvia', 'satureja', 'sinapis', 'thymbra', 'urginea', 'vitis']


In [10]:
label_mapping = {test_class_names[i]: categories.index(test_class_names[i]) for i in range(len(test_class_names))}
label_mapping
print(f"Size label_mapping: {len(label_mapping)}")
print(label_mapping)

Size label_mapping: 16
{'asphodelus': 0, 'castanea': 2, 'ceratonia': 3, 'ebenus': 4, 'erica': 5, 'eucalyptus': 6, 'ferula': 7, 'olea': 9, 'origanum': 10, 'pistacia': 13, 'salvia': 14, 'satureja': 15, 'sinapis': 16, 'thymbra': 17, 'urginea': 18, 'vitis': 19}


In [11]:
y_true_mapped = np.array([label_mapping[test_class_names[i]] for i in y_true])
print(f"Size y_true_mapped: {len(y_true_mapped)}")
print(y_true_mapped)

Size y_true_mapped: 210
[ 0  0  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3
  3  3  3  3  3  3  3  3  3  4  4  4  5  6  7  7  7  7  7  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9 10 10 10 10 10 10 10 10 10 10 10 10 13 13 13 14 14 14 14 14 14 14
 14 14 14 15 15 15 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16
 16 16 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19 19
 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19]


In [12]:
predictions = model.predict(test_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
print(f"Size y_pred: {len(y_pred)}")
print(y_pred)

2025-02-21 14:17:01.402157: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


7/7 [==============================] - 4s 186ms/step
Size y_pred: 210
[ 0  0  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3
  3  3  3  3  3  3  3  6  3  4  4  4  5  6  7  7  7  7  7  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9 10  3 10 10 10 10 10 10 10 10 10 10 13 13 13 14 14 14 14 14 14 14
 14 14 14 15 15  3 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16
 16 16 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 18 19  6 19 19 19 19
 19 19 19 19 19 19 19 19 19 19  3 19 19 19 19 19 19 19]


In [13]:
# Extraindo confiança correta
confidences = [np.max(pred) for pred in predictions]

In [14]:
filenames = test_generator.filenames
df = pd.DataFrame(filenames, columns=['file'])
df['y_true'] = y_true_mapped
df['y_pred'] = y_pred
df['confidence'] = confidences
df['true_label'] = [categories[i] for i in y_true]
df['predicted_label'] = [categories[i] for i in y_pred]
df['status'] = df.apply(lambda row: 'Correct' if row['y_true'] == row['y_pred'] else 'Incorrect', axis=1)

In [15]:
df

,file,y_true,y_pred,confidence,true_label,predicted_label,status
0,asphodelus/Asp.A.ramX400wF(1984)(0)A-10.png,0,0,0.999997,asphodelus,asphodelus,Correct
1,asphodelus/Asp.A.ramX400wF(1987)(0)D-1.png,0,0,0.996568,asphodelus,asphodelus,Correct
2,castanea/Fag.C.satX400wF(781)(0)A-2.png,2,2,0.999803,calicotome,castanea,Correct
3,castanea/Fag.C.satX400wF(781)(0)A-22.png,2,2,0.999874,calicotome,castanea,Correct
4,castanea/Fag.C.satX400wF(781)(0)A-23.png,2,2,0.999926,calicotome,castanea,Correct
...,...,...,...,...,...,...,...
205,vitis/Vit.V.viniX400wF(4984)(0)G-37.png,19,19,0.999554,satureja,vitis,Correct
206,vitis/Vit.V.viniX400wF(4984)(0)G-7.png,19,19,0.999884,satureja,vitis,Correct
207,vitis/Vit.V.viniX400wF(4984)(0)G-9.png,19,19,0.999678,satureja,vitis,Correct
208,vitis/Vit.V.viniX400wF(4985)(0)H-10.png,19,19,0.999877,satureja,vitis,Correct


In [17]:
present_labels = sorted(set(y_true_mapped))  
print(f"Size present_labels: {len(present_labels)}")
print(present_labels)

Size present_labels: 16
[0, 2, 3, 4, 5, 6, 7, 9, 10, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
report = classification_report(y_true_mapped, y_pred, labels=present_labels, 
                                   target_names=[categories[i] for i in present_labels], 
                                   output_dict=True, zero_division=0)

In [20]:
df_report = pd.DataFrame(report).transpose()
print(df_report)

              precision    recall  f1-score    support
asphodelus     1.000000  1.000000  1.000000    2.00000
castanea       1.000000  1.000000  1.000000   21.00000
ceratonia      0.750000  0.900000  0.818182   10.00000
ebenus         1.000000  1.000000  1.000000    3.00000
erica          1.000000  1.000000  1.000000    1.00000
eucalyptus     0.333333  1.000000  0.500000    1.00000
ferula         1.000000  1.000000  1.000000    5.00000
olea           1.000000  1.000000  1.000000   79.00000
origanum       1.000000  0.916667  0.956522   12.00000
pistacia       1.000000  1.000000  1.000000    3.00000
salvia         1.000000  1.000000  1.000000   10.00000
satureja       1.000000  0.666667  0.800000    3.00000
sinapis        1.000000  1.000000  1.000000   20.00000
thymbra        1.000000  1.000000  1.000000    5.00000
urginea        1.000000  1.000000  1.000000   11.00000
vitis          1.000000  0.916667  0.956522   24.00000
accuracy       0.976190  0.976190  0.976190    0.97619
macro avg 

# Matrix

In [21]:
absent_labels = sorted(set(range(len(categories))) - set(present_labels))  
print(f"Size absent_labels: {len(absent_labels)}")
print(absent_labels)

Size absent_labels: 4
[1, 8, 11, 12]


In [22]:
cm = confusion_matrix(y_true_mapped, y_pred, labels=present_labels)
print(cm)

[[ 2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 79  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0 11  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 20  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 11  0]
 [ 0  0  1  0  0  1  0  0  0  0  0  0  0  0  0 22]]
